In [ ]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import FewShotChatMessagePromptTemplate, ChatPromptTemplate, MessagesPlaceholder

llm = ChatOpenAI(
    temperature=0.7,
    streaming=True,
    verbose=True
)

memory = ConversationBufferWindowMemory(
    llm=llm,
    k=2,
    max_token_limit=100,
    memory_key="history",
    return_messages=True
)

examples = [
    {
        "movie": "Top Gun",
        "output": "🛩️👨‍✈️🔥"
    },
    {
        "movie": "The Godfather",
        "output": "👨‍👨‍👦🔫🍝"
    }
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Describe a Movie {movie} only with 3 Emojis"),
        ("ai", "{output}")
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt
)

# few_shot_prompt를 시스템 프롬프트에 통합
system_prompt = (
    "You are an AI function that describes a movie with only 3 Emojis. DO NOT ADD ANYTHING ELSE. "
    "For reference, use the following examples:\n"
)
# 예제를 시스템 프롬프트에 추가
for example in examples:
    system_prompt += f"Movie: {example['movie']} -> {example['output']}\n"

system_prompt += 'if i ask about movie, then reply only 3 emojis. but if i ask about history, then answer like index: Movie -> 3 emoji output'


final_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', system_prompt),
        MessagesPlaceholder(variable_name="history"),
        ('human', '{movie}')
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


# few_shot_prompt는 체인에서 제거하고, 시스템 프롬프트에 예제 포함
final_chain = RunnablePassthrough.assign(
    history=load_memory) | final_prompt | llm

EXAMPLE_MOVIES = [example["movie"] for example in examples]


def invoke_chain(movie):
    result = final_chain.invoke({"movie": movie})

    # 새로운 영화만 메모리에 저장
    if movie not in EXAMPLE_MOVIES:
        memory.save_context(
            {"input": movie},
            {"output": result.content}
        )
    return result.content

In [129]:
invoke_chain("Moonrise Kingdom")

'🏝️👦🏼👧🏻'

In [130]:
invoke_chain("Pulp Fiction")

'🕶️🍔💉'

In [131]:
invoke_chain("What movies that i questioned before?")

'1: Moonrise Kingdom -> 🏝️👦🏼👧🏻\n2: Pulp Fiction -> 🕶️🍔💉'